<a href="https://colab.research.google.com/github/hscrown/seoul_pr/blob/main/weather.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 패키지 임포트

In [328]:
!pip install lxml
!pip install datasets
!pip install haversine

import requests
import pandas as pd
import numpy as np

from datetime import datetime, timedelta
from lxml import etree
from haversine import haversine

# 함수 정의
반드시 순서대로 호출할 것

오늘 날짜 불러와서 문자열로 나타내는 함수

In [329]:
def whats_the_date():
  what_date = datetime.now().strftime("%Y%m%d")
  what_time = datetime.now().strftime("%H%M")
  return what_date, what_time

ip로 현재위치 가져오고 내 위치 정보 출력하기

In [330]:
# ip 주소로 내위치가져오기
def print_ip_info():
  try:
      response = requests.get("http://ipinfo.io")
      data = response.json()
      print("Your IP:", data['ip'])
      print("Your city:", data['city'])
      print("Your region:", data['region'])
      print("Your country:", data['country'])
      print("Your loc (Latitude and Longitude):", data['loc'])
  except Exception as e:
      print("Error:", e)

# 파싱한 데이터는 딕셔너리 형태로 좌표 뿐 아니라 ip주소와 국가와 시 까지 불러올 수 있다.

# 데이터 형태 :  {'ip': '1.226.221.109', 'city': 'Seoul', 'region': 'Seoul', 'country': 'KR', 'loc': '37.5660,126.9784', 'org': 'AS9318 SK Broadband Co Ltd', 'postal': '03141', 'timezone': 'Asia/Seoul', 'readme': 'https://ipinfo.io/missingauth'}

내 위치정보 딕셔너리에서 내 좌표정보만 가져오기

In [331]:
def get_my_loc(info):
  my_lat, my_lon = my_coor['loc'].split(',')
  my_lat = float(my_lat)
  my_lon = float(my_lon)
  my_loc = (my_lat, my_lon)

  return my_loc

기상청 좌표 데이터 전처리함수

In [332]:
# 데이터셋 라이브러리를 사용하여 기상청 데이터셋 로드 - 로컬에서 안불러와도됨
def bring_weather_coor():
  dataset = load_dataset("hscrown/weather_api_info")
  kor_loc = pd.DataFrame(dataset['train'])
  # kor_loc = pd.read_csv('dataset.csv')
  kor_loc = kor_loc.iloc[:,:15] # 필요한 컬럼만 추출
  kor_loc = kor_loc.dropna() # 2단계와 3단계가 모두 존재하는 행만 추출

내 좌표를 넣으면 가장 가까운 기상청 x,y좌표로 변환하는 함수

In [333]:
# 내 좌표를 기상청 x,y좌표로 변환

def find_nearest_grid(my_loc):
  grid = None
  min_distance = float('inf')  # 무한대 값으로 초기화

  for index, row in kor_loc.iterrows():
      # 각 격자 지점에 대한 튜플 (위도, 경도)을 생성
      grid_point = (row['위도(초/100)'], row['경도(초/100)'])

      # haversine 공식을 사용하여 거리계산
      distance = haversine(my_loc, grid_point)

      # 가장 가까운 거리를 찾으면 정보를 업데이트
      if distance < min_distance:
          min_distance = distance
          grid = row
          nx = grid['격자 X']
          ny = grid['격자 Y']

      return grid,nx,ny

내 위치와 가장가까운 공원, 박물관 찾아주는 함수정의

In [334]:
# 내 위치와 가장 가까운 공원찾는 함수 정의
def find_nearest_place(my_loc,df):
  grid = None
  min_distance = float('inf')  # 무한대 값으로 초기화

  for index, row in df.iterrows():
      # 각 격자 지점에 대한 튜플 (위도, 경도)을 생성
      grid_point = (row['LATITUDE'], row['LONGITUDE'])

      # haversine 공식을 사용하여 거리계산
      distance = haversine(my_loc, grid_point)

      # 가장 가까운 거리를 찾으면 정보를 업데이트
      if distance < min_distance:
          min_distance = distance
          result = row

      return result

# 좌표와 시간을 넣으면 기상정보를 제공하는 함수

In [335]:
def get_weather_info(base_date,base_time,nx,ny):

  # 초단기실황데이터
  url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtNcst'
  params ={
      'ServiceKey': 'sX3JWddMWHJxC43fx9mqgcqSsbmAlTpoFTUPbnrE1Db5uVnEAs7gJIL4Z3tzW1u2S6UC+8/go3xYCnG2wDctAQ==',
      'pageNo': '1',
      'numOfRows': '1000',
      'dataType': 'XML',
      'base_date': base_date,
      'base_time': base_time,
      'nx': nx,
      'ny': ny
  }

  response = requests.get(url, params=params)
  root = etree.fromstring(response.content)

  # 데이터 파싱 및 추출
  # category = root.xpath('//category/text()')[0] # 0:강수형태, 2:습도, 3:기온, 4:풍속
  rain = root.xpath('//obsrValue/text()')[0] # 강수
  temp = root.xpath('//obsrValue/text()')[3] # 기온

  return rain,temp

# print(f"Result Code: {resultCode}, Message: {resultMsg}")
# print(f"Number of Rows: {numOfRows}, Page Number: {pageNo}, Total Count: {totalCount}")
# print(f"Data Type: {dataType}, Base Date: {base_date}, Base Time: {base_time}")
# print(f"Coordinates: ({nx}, {ny}), Category: {category}, Observed Value: {obsrValue}")

# 공원데이터 전처리 코드

In [336]:
def get_park_data():
  # API 요청
  start_point = 1
  end_point = 1000 # 최대 1000개까지만 호출 할 수 있음
  seoul_key = '57524f76506d656e3732636a52457a'


  url = f'http://openAPI.seoul.go.kr:8088/{seoul_key}/json/SearchParkInfoService/{start_point}/{end_point}/'

  park = requests.get(url).json()
  park.keys() # ['SearchParkInfoService']

  park = pd.DataFrame(park['SearchParkInfoService']['row'])

  # ['LATITUDE'] 컬럼을 실수로 변경
  # 결측행 삭제
  park['LATITUDE'].replace('', np.nan, inplace=True)
  park['LONGITUDE'].replace('', np.nan, inplace=True)
  park = park.dropna()

  # ['LATITUDE'] 컬럼을 실수로 변경
  park['LATITUDE'] = park['LATITUDE'].astype(float)

  # ['LONGITUDE'] 컬럼을 실수로 변경
  park['LONGITUDE'] = park['LONGITUDE'].astype(float)

  return park

# 실행 코드

In [337]:
# 변수에 오늘날짜 담기
base_date, base_time = whats_the_date()
base_date, base_time

('20240422', '1958')

In [338]:
# 내 위치정보
my_info = print_ip_info()
my_info

Your IP: 34.168.44.135
Your city: The Dalles
Your region: Oregon
Your country: US
Your loc (Latitude and Longitude): 45.5946,-121.1787


In [339]:
# 변수에 내 위치 담기
my_info= {'ip': '1.226.221.109', 'city': 'Seoul', 'region': 'Seoul', 'country': 'KR', 'loc': '37.5660,126.9784', 'org': 'AS9318 SK Broadband Co Ltd', 'postal': '03141', 'timezone': 'Asia/Seoul', 'readme': 'https://ipinfo.io/missingauth'}
my_info
my_loc = get_my_loc(my_info)
my_loc

(37.566, 126.9784)

In [340]:
# 내 위치와 가장 가까운 위치 찾기
grid,nx,ny = find_nearest_grid(my_loc)

In [341]:
grid[['1단계','2단계','3단계']]

1단계    서울특별시
2단계      종로구
3단계    청운효자동
Name: 2, dtype: object

In [342]:
nx,ny

(60, 127)

In [343]:
rain, temp = get_weather_info(base_date,base_time,nx,ny)
rain, temp

('0', '21.5')

In [352]:
park = get_park_data()
park.head(1)

P_IDX    P_PARK                                     P_LIST_CONTENT  \
0     1  남산도시자연공원  남산공원은 도심에 위치하여 서울시민에게 맑은 공기를 제공하는 자연휴식처이며 산책 꽃...   

                                           AREA    OPEN_DT  \
0  2896887㎡ 임 야 : 2454140㎡ 녹지대 및 기타시설 : 442747㎡  1968.9.10   

                                          MAIN_EQUIP  \
0  기반시설 : 광장 45950㎡ 도로 108530㎡ 산책로 6.7㎞ (북측:3.7㎞/...   

                         MAIN_PLANTS  \
0  소나무 단풍 아카시아 상수리나무 등 191종 2881870주   

                                            GUIDANCE  \
0  http://parks.seoul.go.kr/upload/seditorMulti/2...   

                                          VISIT_ROAD  \
0  명동역(남산케이블카 와룡묘 서울애니메이션센터 방면) 도보 명동역 3번출구 → 퍼시픽...   

                                           USE_REFER  \
0  남산공원 차량통행 금지안내 2005년 5월 1일부터 남산공원에 일반승용차 택시 통행...   

                                               P_IMG P_ZONE  \
0  http://parks.seoul.go.kr/file/info/view.do?fId...     중구   

                   P_ADDR          P_NAME    P_ADMINTEL G_LONGITUDE  \
0  서울특별시 중구 삼일대로 231(예장동)  서울특별시중부푸른도시사업소  02-3783-5900  198364.107   

   G_LATITUDE   LONGITUDE  LATITUDE  \
0  450395.554  126.990377  37.55014   

                                      TEMPLATE_URL  
0  http://parks.seoul.go.kr/template/sub/namsan.do

In [365]:
# 강수가 맑음이 아니거나 기온이 30도 이상이면 도서관이나 미술관에 가세요
def where_to_go(rain,temp):
  print(f'오늘 날씨는 "{rain}", 기온은 {temp}도 입니다.')
  if (temp == 0 or float(rain) >= 30) :
     return '도서관이나 미술관에 가세요'
  else: return '공원에 가도 좋습니다', find_nearest_place(my_loc,park)['P_PARK']

In [364]:
where_to_go(rain,temp)

오늘 날씨는 0, 기온은 21.5도 입니다.


('공원에 가도 좋습니다', '남산도시자연공원')